In [1]:
## Chapter04から使用するテキストデータをコピーする（事前にChapter04/Sentiment.ipynbを実行してください）
!cp -r ../../Chapter04/sisyou_db ./

In [2]:
import spacy
import pandas as pd
import numpy as np


## モデルのロードとデータの読み込み
nlp = spacy.load("ja_ginza")
df = pd.read_excel("sisyou_db/sisyou_db_h29_01.xlsx", skiprows=[1])
df.head()

,ID,年号,年,月,発生\n時間,災害状況,業種（大分類）,Unnamed: 7,業種（中分類）,Unnamed: 9,...,事業場\n規模,起因物（大分類）,Unnamed: 14,起因物（中分類）,Unnamed: 16,起因物（小分類）,Unnamed: 18,事故の型,Unnamed: 20,年齢
0,1,平成,29,1,22～23,工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前...,1,製造業,108,化学工業,...,―,3,その他の装置等,37,用具,379,その他の用具,3,激突,54
1,2,平成,29,1,19～20,倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って...,8,商業,804,その他の商業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,39
2,3,平成,29,1,18～19,会社構内にて車輌の洗車中、足を滑らせ転倒した際に左手をつき、翌朝に左肩の痛みが大きくなり、左...,4,運輸交通業,403,道路貨物運送業,...,100～299,7,環境等,71,環境等,719,その他の環境等,2,転倒,54
3,4,平成,29,1,16～17,厩舎2階でバッカン受け入れ作業中、バッカンを落とす穴から落下した。,7,畜産・水産業,701,畜産業,...,10～29,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,414,開口部,1,墜落、転落,50
4,5,平成,29,1,16～17,勤務先の食堂施設内で、ダンボールを束ねてビニールの荷造り紐で縛り結んだ時、手が滑り勢いよく壁...,8,商業,802,小売業,...,50～99,4,仮設物、建築物、構築物等,41,仮設物、建築物、構築物等,419,その他の仮設物、建築物、構築物等,6,激突され,65


In [3]:
## 文書ベクトルの表示
text = df["災害状況"][0]
print(text)

doc = nlp(text)
print(doc.vector)

工場内で床に置いていたコードに、荷物を抱えていた状態のときに足が引っ掛かり、よろめいて数歩前に進んだのち、前方にあった作業台に衝突して受傷した。
[ 5.42991329e-03 -1.25486106e-01  3.05590015e-02 -9.48476791e-02
 -8.41306597e-02  2.19089966e-02 -6.87961727e-02 -5.37309684e-02
 -1.63278595e-01 -7.78466314e-02  5.82386814e-02 -9.85572636e-02
 -9.53303557e-03  3.43594104e-02 -1.18678160e-01 -1.82270318e-01
 -9.89774540e-02 -8.99542272e-02 -8.26774091e-02  5.89721426e-02
 -8.77029449e-02  1.12206280e-01  9.54070594e-03  7.47752935e-02
 -9.60959122e-02 -1.12004898e-01 -1.80774570e-01  4.85713035e-02
  5.92021085e-03  4.22824249e-02 -2.48667412e-03 -6.61683455e-02
  1.49948031e-01  1.32860467e-01 -9.37512703e-03 -1.13776335e-02
 -7.60364160e-02  8.44385847e-02 -6.37349039e-02  1.39465686e-02
  9.07834247e-02  3.96359898e-02 -8.47832412e-02 -6.64764717e-02
 -1.17050312e-01 -6.79016411e-02  6.10791892e-02 -2.60532554e-03
 -1.16309293e-01  7.03809559e-02  2.22203802e-04  2.34428421e-02
  2.61440612e-02  7.45722726e-02  2.74467990e-02 -4.26080264e-02
  2.04467941e-02 

In [4]:
## 文書間類似度の算出
texts = [
    "工場内で、鉄骨階段（仮組）高さ2m50cmに手摺を仮組立作業中、足を滑らせて転倒し、顔面を強打して骨折し、右脇腹肋骨も骨折した。",
    "倉庫の出入口の階段を荷物（冷凍商品15kgぐらい）を持って下りる際に、階段が凍っていて滑って転倒し、階段を転げ落ち（4段位）、持っていた荷物を足に落としてしまい、右足の腓骨を骨折した。",
    "道路の3車線の真ん中を走行中、左車線に侵入してしまい、走行中の大型ワンボックスカーと衝突し、首と左肩を痛め、回転性のめまいで入院し、痺れもある。"
]
docs = list(nlp.pipe(texts))

for i in range(len(docs)-1):
    for j in range(i+1, len(docs)):
        print(i, j, docs[i].similarity(docs[j]))

0 1 0.9340122716925394
0 2 0.9009380391944312
1 2 0.8959438510857466


In [5]:
## 文書間類似度行列の作成
from sklearn.preprocessing import normalize

vecs = [doc.vector for doc in nlp.pipe(df["災害状況"])]
X = np.array(vecs)
X = normalize(X, axis=1, norm='l2')

cos_sims = X @ X.T
print(cos_sims)

[[0.9999996  0.9579886  0.91829884 ... 0.89780945 0.9683267  0.94647825]
 [0.9579886  0.99999994 0.9257992  ... 0.8828778  0.9620715  0.93991107]
 [0.91829884 0.9257992  0.9999999  ... 0.92673296 0.9437607  0.9205553 ]
 ...
 [0.89780945 0.8828778  0.92673296 ... 0.9999995  0.9129927  0.9005907 ]
 [0.9683267  0.9620715  0.9437607  ... 0.9129927  1.         0.9469491 ]
 [0.94647825 0.93991107 0.9205553  ... 0.9005907  0.9469491  1.0000002 ]]


In [6]:
## 類似度行列を用いて類似文書検索
target = 9
print(target, df["災害状況"][target])

top_index = np.argsort(cos_sims[target])[::-1]
print(df["災害状況"][top_index][1:10]) #top_index[0]は入力文書

9 棚卸作業をしていたところ、ラムダテナー（資材用ラック）2段目にかけた梯子に足を掛けた際に、梯子がずれ落ちてバランスを崩し、右片足で着地した。
外傷は特に無かったが、右足を地面に着けて立ち上がる事が出来なくなり、右足アキレス腱断裂と診断された。
1756    売場業務中、売場外から中へ入る時、レジ横小扉を開け、中に入ろうとした時、扉の下側に左足が引っ...
800     \n左足を軽トラックの荷台に載せ、右足をフロアにつけた状態で作業をしていたが、工具箱がフロア...
1425    鮮魚作業場にて、うなぎをパック詰め作業をしている際、タレを取りにいこうと冷蔵庫に向かったとこ...
2099    プラスト工場で、ボックス（2m弱）によりプラスト作業中、脚立の後方にあるサンドブラストホース...
2693    塗装場所へ移動する為、5尺の脚立をはしご状態にして、約2.3m程上がった屋上へ上る途中に使用...
305     荷卸し終了後にトラックの荷台より降りようとした際、足を滑らせて地面へ転倒し、左腰後ろの部分を...
2213    1番ホールド内で脚立、ローリングタワーが使用不可能な場所でパイプ上に乗り、養生はずしの作業を...
153     事業所内2階から1階へ階段を下りたところ、階段のすぐ前に一時的に置いてあった使用済みタオル袋...
1477    30kgくらいの荷物を3階へ運んでいる際に足場が確認できず、不安定な状態で足が着地したために...
Name: 災害状況, dtype: object
